# init

In [1]:
import matplotlib.pyplot as plt

%matplotlib inline

# ссылки

Шпаргалка для технического собеседования: https://habr.com/ru/company/vk/blog/350326/

# Sort

In [1]:
a = [1,5,2,4,3]

a.sort()

a

[1, 2, 3, 4, 5]

In [3]:
a.sort(reverse=True)

a

[5, 4, 3, 2, 1]

In [4]:
b = a

a.sort()

b # sorted

[1, 2, 3, 4, 5]

In [5]:
b = a.copy() # copy !

a.sort(reverse=True)

b # not sorted

[1, 2, 3, 4, 5]

In [6]:
a

[5, 4, 3, 2, 1]

# Два/три указателя

Метод двух указателей работает, когда мы имеем дело с монотонными функциями.  
> Функция называется монотонно возрастающей, если каждое её значение строго больше предыдущего. Функция называется монотонно неубывающей, если каждое её значение не меньше предыдущего. Аналогично функция может быть монотонно убывающей (каждое значение строго меньше предыдущего) или монотонно невозрастающей (каждое значение не больше предыдущего). Если функция обладает хотя бы одним из этих свойств, то она монотонная.

**Off-by-one** — это ошибка в индексе на 1. Это самый распространённый класс ошибок в таких задачах (на два указателя).

Пример задачи - найти максимальное число одинаковых подряд идущих символов в строке:

In [8]:
# tests
tests = {
    'aababbb': 3,
    'aababb': 2,
    'aaababa': 3,
    'abcde': 1,
    '': 0,
    'aaa': 3
}

In [9]:
# two pointers, O(n):
def max_consecutive_elements(input_str):
    result = 0
    cur_idx = 0
    while cur_idx < len(input_str):
        next_idx = cur_idx
        while next_idx < len(input_str) and input_str[next_idx] == input_str[cur_idx]:
            next_idx += 1
            
        result = max(result, next_idx - cur_idx)
        cur_idx = next_idx
        
    return result

In [10]:
for input_v, result_v in tests.items():
    assert max_consecutive_elements(input_v) == result_v

Мы тут справляемся за $O(n)$, потому что не пробегаем два раза ни по одному символу в строке: если последовательность закончилась, мы сразу ставим `cur_idx` в её конец (`next_idx`).  

А вот если бы использовали цикл `for`, т.е. прошлись бы переменной `cur_idx` по каждому символу, и потом бы от него замеряли длину последовательности, то получилось бы $O(n\log{n})$ в среднем, и $O(n^2)$ в худшем случае.

# Бинарный поиск

Вспомним реализацию метода на примере классической задачи. Есть _упорядоченный_ массив целых чисел, нужно определить, есть ли в нём число `x`.

In [13]:
# tests
tests = [
    {'input': ([1,2,3], 3), 'result': True},
    {'input': ([], 3), 'result': False},
    {'input': ([1,2,3,4,5], 3), 'result': True},
    {'input': ([1], 1), 'result': True},
    {'input': ([1,2,3], 4), 'result': False},
]

In [14]:
def binary_search(arr, target):
    left_idx = 0
    right_idx = len(arr)
    
    while left_idx < right_idx:
        mid_idx = (left_idx + right_idx) // 2
        
        if arr[mid_idx] == target:
            return True
        elif arr[mid_idx] < target:
            left_idx = mid_idx + 1
        else:
            right_idx = mid_idx
            
    return False 

In [23]:
for test_dict in tests:
    assert binary_search(*test_dict['input']) == test_dict['result']

Здесь сложность - $O(\log{n})$. В общем и худшем случае интервал поиска на каждом шаге алгоритма будет сокращаться в два раза. Изначально он равен $n$, после одной итерации он будет содержать $\frac{n}{2}$ элементов, затем $\frac{n}{4}$, на $k$-й итерации в нём будет $\frac{n}{2^k}$ элементов. Как только $2^k$ станет больше $n$, алгоритм завершится.

Этот метод позволяет искать не только элемент массива. Пусть нам нужно найти решение уравнения $x \cdot \log_2(x) = Y$. Вычислить значение выражения $x\cdot \log_2(x)$ легко, а найти решение уравнения математически — нет. Поскольку функция $x\cdot \log_2(x)$ монотонно возрастает при $x \geqslant 1$, можно применить бинарный поиск.

In [31]:
def equation(x):
    return x * log2(x)

In [32]:
from math import log2

def solve_equation(value):
    left, right = 1, value
    for _ in range(100):
        mid = (left + right) / 2
        expr_result = mid * log2(mid)
        if expr_result < value:
            left = mid
        else:
            right = mid
    return mid 

Здесь вместо сравнения left и right с заданной точностью мы применили фиксированное число итераций (100). Это гарантирует, что числа будут очень близки. При этом не нужно думать о том, как правильно подбирать точность сравнения.

In [37]:
y = 10

x = solve_equation(y)

x

4.564956833533316

In [38]:
equation(x) # должно быть примерно равно y

9.999999999999996

В неупорядоченном массиве применить бинарный поиск, чтобы найти элемент, не получилось бы. Как правило, бинарный поиск применяется на монотонных данных. Но некоторые задачи формально не удовлетворяют условию монотонности и, тем не менее, решаются бинарным поиском. Например:  
> дана бинарная строка длины N, состоящая только из нулей и единиц. Гарантируется, что самый левый её элемент 0, а самый правый — 1. Найдите любое вхождение подстроки “01”.  

Будем рассматривать разные подстроки и поддерживать инвариант, что самый левый символ равен нулю, а самый правый — единице. Изначально подстрока равна строке и инвариант выполняется по условию задачи. На очередном шаге рассмотрим середину подстроки: если это 0, то на следующем шаге подстрокой будет всё от этого символа до конца текущей подстроки. Аналогично, если середина равна 1, то возьмём левую часть подстроки. Таким образом, инвариант продолжает выполняться, а длина подстроки уменьшилась вдвое за одну итерацию.

In [51]:
def find_substring_binsearch(bin_string: str):
    left_idx = 0
    right_idx = len(bin_string)
    
    while left_idx < right_idx:
        mid_idx = (left_idx + right_idx) // 2
        
        if bin_string[mid_idx] == '0':
            left_idx = mid_idx
        elif bin_string[mid_idx] == '1':
            right_idx = mid_idx
        else:
            raise ValueError()
        
        if (right_idx - left_idx) == 1:
            break
        
    return left_idx, right_idx

In [53]:
s = '010010101001011001001001001'

left_idx, right_idx = find_substring_binsearch(s)

left_idx, right_idx

(3, 4)

In [54]:
s[left_idx]+s[right_idx]

'01'

Как и в методе двух указателей, в бинарном поиске легко допустить незаметную ошибку в индексах. 

> найти индекс максимального элемента _упорядоченного_ массива arr, не превосходящего X. Если такого элемента не существует, вернуть -1.

In [73]:
def find_max_index(arr, x):
    if len(arr) < 1:
        return -1
    
    result = -1
    
    left_idx = 0
    right_idx = len(arr)
    
    while left_idx < right_idx:
        mid_idx = (left_idx + right_idx) // 2
        
        if arr[mid_idx] > x:
            right_idx = mid_idx
        elif arr[mid_idx] <= x:
            left_idx = mid_idx
            result = max(result, left_idx)
        else:
            raise ValueError()
            
        if (right_idx - left_idx) == 1:
            break
            
    return left_idx

In [74]:
# tests
tests = [
    {'input': ([1,2,3], 3), 'result': 2},
    {'input': ([], 3), 'result': -1},
    {'input': ([1,2,3,4,5], 0), 'result': 0},
    {'input': ([1], 1), 'result': 0},
    {'input': ([1,2,3], 4), 'result': 2},
]

In [75]:
for test_dict in tests:
    try:
        assert find_max_index(*test_dict['input']) == test_dict['result']
    except:
        print()
        print('Breaks with input', test_dict['input'])
        print('Func result:', find_max_index(*test_dict['input']))
        print('Expected result:', test_dict['result'])

Более элегантное решение:

In [ ]:
def max_lower_or_equal(sorted_arr, value):
    # выносим в начало проверку на существование. Здесь все варианты с -1 (поэтому дальше обходимся без result)
    if not sorted_arr or sorted_arr[0] > value:
        return -1

    left_idx, right_idx = 0, len(sorted_arr)
    while left_idx + 1 < right_idx: # +1 заменяет "if (right_idx - left_idx) == 1: break"
        mid_idx = (left_idx + right_idx) // 2
        if sorted_arr[mid_idx] <= value:
            left_idx = mid_idx
        else:
            right_idx = mid_idx
    return left_idx 

При любой реализации бинарного поиска можно допустить ошибку. Вот как минимизировать вероятность её появления:
* Используйте полуинтервалы (`left = 0`, `right = len(arr)`), а не интервалы (когда правая граница включена). Сформулируйте для себя инвариант. Для задачи выше он будет звучать как “`arr[left]` не превосходит искомый элемент, `arr[right]`, наоборот, превосходит”. Тогда при чтении кода вы можете проверить этот инвариант. Именно поэтому в примере кода выше мы вынесли проверку существования искомого элемента в отдельное условие. Если такого элемента в массиве не существует, то инвариант при инициализации не выполнялся бы и решение было бы некорректным.
* Проверяйте три простых теста:  
    - [1, 2], ответ 1. Для задачи выше это будет arr = [1, 2], X = 1.
    - [1, 2], ответ 2. Для задачи выше это будет arr = [1, 2], X = 2.
    - Ответа нет. Для задачи выше это будет arr = [1, 2], X = 0.

# Рекурсия и поиск с возвратом

Рекурсия — это когда функция вызывает сама себя. Рекурсия — не какой-то конкретный алгоритм, а техника, которая позволяет реализовывать алгоритмы.  

## Задачи, которые решаются итеративно

Самый простой и известный пример — числа Фибоначчи. Они задаются рекуррентным соотношением $f_i = f_{i-1} + f_{i-2}$, начальные условия обычно задаются как $f_0 = 0$, $f_1 = 1$. Задача: написать функцию, которая принимает целое число $n$ и возвращает $n$-ное число Фибоначчи. Можно написать рекурсивное решение, но в нём тогда нужно правильно учесть:
* Начальные условия, чтобы решение не ушло в бесконечный цикл.
* Мемоизацию промежуточных результатов (сохранение результатов промежуточных вычислений), чтобы решение работало за линейное время, а не за экспоненту.

In [ ]:
def nth_fibonacci_recursive(n):
    SENTINEL = -1
    # Переменная для мемоизации
    numbers = [SENTINEL] * (n + 1)

    def nth_fibonacci_(n):
        if n <= 1:
            return n
        elif numbers[n] != SENTINEL:
            return numbers[n]
        else:
            result = nth_fibonacci_(n - 1) + nth_fibonacci_(n - 2)
            numbers[n] = result
            return result

    return nth_fibonacci_(n) 

In [ ]:
def nth_fibonacci_iterative(n):
    if n <= 1:
        return n
    previous, current = 0, 1
    for _ in range(n - 1):
        previous, current = current, previous + current
    return current 

Если вы знаете, как решить задачу итеративно и рекурсивно и сомневаетесь, какой способ лучше использовать — пишите итеративное решение. Так проще избежать ошибок.

## Разбор выражений

Такие задачи не очень популярны на собеседованиях из-за высокой сложности, поэтому не будем разбирать их подробно. Приведём два примера:  
1. Рассмотрим простой алгоритм сжатия строки. Если в строке есть несколько подряд идущих одинаковых подстрок, можно заменить их на группу. Например, строку `aabaabaab` можно записать как `3(aab)`. Можно алгоритм применить к сжатой строке, получив вложенные группы: `3(2ab)`. Дана сжатая строка, требуется её распаковать. Например, для строки `a2(a2(bс))3db` ответ будет `aabcbcabcbcdddb`.
2. Написать калькулятор арифметических выражений, содержащих операции `+, -, *, /`, а также скобки.

## Поиск с возвратом (Backtracking)

Это метод решения задач, где требуется перебор вариантов.  

Рассмотрим задачу: дано число N, нужно сгенерировать все правильные скобочные последовательности из N открывающих и N закрывающих скобок.  

В задачах на полный перебор число возможных вариантов растёт экспоненциально относительно входных данных, поэтому ограничения обычно очень маленькие. Например здесь кол-во вариантов - $O(4^N)$  

При решении переборных задач результат не всегда удобно передавать через возвращаемое значение. Тогда приходится пользоваться аккумуляторами. В таком случае нужно внимательно следить за изменением их состояний.

In [76]:
def generate(n):
    result = []

    def generate_(left_open, left_closed, accum):
        if not left_open and not left_closed:
            result.append(accum)
            return
        if left_open:
            generate_(left_open - 1, left_closed, accum + '(')
        if left_closed > left_open:
            generate_(left_open, left_closed - 1, accum + ')')

    generate_(n, n, '')
    return result 

In [81]:
generate(4)

['(((())))',
 '((()()))',
 '((())())',
 '((()))()',
 '(()(()))',
 '(()()())',
 '(()())()',
 '(())(())',
 '(())()()',
 '()((()))',
 '()(()())',
 '()(())()',
 '()()(())',
 '()()()()']

Это решение хорошее. Скорее всего, на собеседовании его примут. Но на каждом шаге мы копируем строку, поэтому решение можно ускорить в N раз, если использовать и для текущей строки аккумулятор.

In [78]:
def generate(n):
    result = []

    def generate_(left_open, left_closed, accum):
        if not left_open and not left_closed:
            result.append(''.join(accum))
            return
        if left_open:
            accum.append('(')
            generate_(left_open - 1, left_closed, accum)
            accum.pop()
        if left_closed > left_open:
            accum.append(')')
            generate_(left_open, left_closed - 1, accum)
            accum.pop()

    generate_(n, n, [])
    return result 

In [79]:
generate(3)

['((()))', '(()())', '(())()', '()(())', '()()()']

## Рекурсивный обход

В реальной жизни рекурсию очень удобно применять для обхода графов или деревьев — таких как файловая система. Метод называется «поиск в глубину». Мы рассмотрим данный алгоритм более подробно в уроке «Алгоритмы на графах».

## Метод «Разделяй и властвуй» (Divide-and-conquer)

Этот метод работает так:  
* Исходные данные разделяются на несколько частей.  
* Для каждой из частей функция вызывается рекурсивно.  
* Результаты рекурсивных вызовов объединяются.  
Этот метод лежит в основе эффективных алгоритмов сортировки. Им будет посвящен весь следующий урок.

# об асимптотической сложности переборных задач

Задача: создать список всех возможных перестановок чисел от 1 до N. Сколько памяти займёт результирующий список?  
> $O(N \cdot N!)$: Всего существует N! перестановок, каждая — длины N.  
> $O(N^{N+1})$: С точки зрения порядка роста, это то же самое, что и $O(N \cdot N!)$. Факториал — самая быстрорастущая функция, которая может вам встретиться в задачах на собеседованиях.

# Сортировки

На собеседованиях редко просят написать именно алгоритм сортировки, кандидат может пользоваться функциями сортировки из стандартной библиотеки языка. Поэтому достаточно знать:
* Время работы различных алгоритмов сортировок в среднем и худшем случае.
* Затраты по памяти.
* Общую идею реализации хотя бы одного быстрого алгоритма.
* Как вызвать сортировку в вашем языке программирования.  

Начнём с того, что вспомним главную идею основных алгоритмов сортировки:  
* **Вставками**: Элементы по очереди, слева направо, добавляются в отсортированную часть массива.  
* **Слиянием**: Массив делится пополам, сортировка рекурсивно вызывается для обеих частей, затем создаётся временный массив, в который по порядку складываются значения из обеих частей.  
* **Быстрая**: Выбирается элемент. Все меньшие элементы переносятся в левую часть, бо́льшие — в правую. Затем алгоритм вызывается рекурсивно от левой и правой частей.  
* **Пирамидальная**: Используется вспомогательная структура данных, которая позволяет быстро находить максимальный элемент.  

Мы вынесли информацию по затратам в сводную таблицу:  

| Сортировка    | Время работы в среднем/худшем  | Затраты по памяти в среднем/худшем |  
|---------------|--------------------------------|------------------------------------|  
| Вставками     | $O(n^2)$/$O(n^2)$              | $O(1)$/$O(1)$                      |  
| Быстрая       | $O(n \log n)$/$O(n^2)$         | $O(\log n)$/$O(n)$                 |  
| Слиянием      | $O(n \log n)$/$O(n \log n)$    | $O(n)$/$O(n)$                      |  
| Пирамидальная | $O(n \log n)$/$O(n \log n)$    | $O(1)$/$O(1)$                      |  

Обратите внимание, что быстрая сортировка в худшем случае может работать плохо, но вероятность этого крайне мала. Чтобы гарантировать быстрое выполнение даже в худшем случае, существуют комбинации алгоритмов, например, **интроспективная сортировка, Introsort**. Основа алгоритма — быстрая сортировка с модификациями:
* Если глубина рекурсии превышает пороговое значение, зависящее от log n, то алгоритм использует пирамидальную сортировку.
* Если на текущем шаге нужно отсортировать небольшое количество элементов, то используется сортировка вставками.  

На практике в основном используются только алгоритмы со временем работы $O(n\log n)$, кроме случаев очень маленьких n.  

**Python** юзает [Timsort](https://ru.wikipedia.org/wiki/Timsort): `arr.sort()` или `sorted_arr = sorted(arr)`

# Алгоритмы на графах

На собеседованиях в крупные зарубежные компании графы и деревья — самые популярные темы. Несмотря на то, что деревья — частный случай графов, задачи сильно различаются, поэтому мы рассмотрим их в разных уроках. Начнём с графов.  

Граф состоит из множества вершин $V$ и множества рёбер $E$. Каждое ребро соединяет пару вершин из $V$. По аналогии с обозначениями из теории множеств пишут, что граф содержит $|V|$ вершин и $|E|$ рёбер.  

Задачи на графы с алгоритмических собеседований можно классифицировать по трём признакам:  
* типу графа,  
* способу задания,  
* что нужно в этом графе найти.  

## Типы графов

Графы — очень богатая тема, поэтому можно привести много классов графов по разным измерениям. Мы ограничимся только тем, что важно в контексте алгоритмических собеседований.  

### По наличию направления

Графы бывают ориентированные (направленные) и неориентированные (ненаправленные).  

В первом случае наличие ребра $(v, u)$ означает, что по этому ребру можно пройти из вершины $v$ в вершину $u$. Существование обратного ребра не гарантируется. Аналог из реальной жизни: дорога с односторонним движением.  

В неориентированном графе по любому ребру можно пройти в обе стороны.  

Технически стандартные способы задания графов поддерживают только ориентированные рёбра. Поэтому, чтобы задать неориентированный граф, обычно для каждого ребра $(v, u)$ заводят два ориентированных: $(v, u)$ и $(u, v)$. Аналогия из предыдущего пункта работает и в этом случае: если по дороге можно двигаться в обе стороны, то можно считать, что у нас две разные дороги в зависимости от направления.  

Если тип графа неясен из условия, спрашивайте интервьюера, чтобы не упустить этот нюанс.

**Задача:** дан набор слов, стартовое и конечное слово. За один ход можно взять текущее слово и заменить его на любое другое из этого набора, если они отличаются ровно на один символ. Например, дан набор
`[”cat”, “cap”, “tab”, “tap”]`, начальное слово `“cat”`, конечное — `“tap”`. Можно совершить цепочку преобразований `“cat” → “cap” → “tap”`, а
`“cap” → “tab” → “tap”` нельзя, поскольку `“cap”` и `“tab”` отличаются не в одном символе.  
За какое наименьшее число ходов можно превратить стартовое слово в конечное?

### По наличию весов рёбер

Рёбра графов могут иметь веса, тогда граф называется взвешенным. Можно считать вес штрафом, который нужно заплатить за проход по ребру. В реальной жизни таким весом могла бы быть длина дороги или время, за которое по ней можно проехать. Обычно мы хотим найти минимальный штраф (расстояние или время), чтобы добраться из одной вершины в другую. Бывают и другие задачи для взвешенных графов, но они почти не встречаются на собеседованиях.

### По наличию циклов

Говорят, что в графе есть цикл, если можно стартовать в некоторой вершине и вернуться в неё после перемещения по рёбрам, не проходя дважды по одному ребру.  

Граф называется ациклическим, если в нём нет циклов. Если граф при этом неориентированный и связный, он называется «дерево». Про алгоритмы на деревьях мы поговорим в следующем уроке.  

> К ориентированным графам неприменимо понятие «дерево». Если в ориентированном графе нет цикла, то он так и называется — ациклический ориентированный граф. Иногда можно встретить аббревиатуру DAG — directed acyclic graph.  

### вероятные задачи

На собеседовании могут встретиться задачи:
* Определить, есть ли цикл в ориентированном графе.
* Определить, есть ли цикл в неориентированном графе.
* Найти топологическую сортировку вершин ориентированного графа.  

Если у вас есть свободное время — можете повторить соответствующие алгоритмы. Если времени не очень много — лучше потратить его на другие темы, поскольку данные задачи встречаются не очень часто.

## Представления графов

Чтобы представить граф в памяти, его можно явно задать с помощью списка рёбер, матрицы смежности или списка смежных вершин. Это классические способы, их обычно разбирают при изучении алгоритмов и структур данных, поэтому не будем на них подробно останавливаться.  

Другой популярный на собеседованиях вариант — описание карты и правил движения по ней. Пример: дано описание поля размера $N\times M$ с помощью массива строк одинаковой длины, в котором символ точка $‘.’$ означает свободную клетку, а решётка $‘#’$ — занятую. Из свободной клетки за один ход можно переместиться в любую соседнюю по стороне свободную клетку. Нужно определить длину кратчайшего пути из левого верхнего угла в правый нижний. Если такого пути не существует, вернуть $-1$. Выходить за пределы поля нельзя.  
В таких задачах часто проще обойтись без стандартных способов представления, а придумать свой.

## Алгоритмы на графах

На собеседованиях время сильно ограничено, поэтому задачи требуют только базовых навыков. Обычно в задаче требуется сделать что-то из следующего:  
* Проверить существование пути из одной вершины в другую или определить, является ли неориентированный граф связным.  
* Найти кратчайший путь в невзвешенном графе.  
* Найти кратчайший путь во взвешенном графе.  

Первая задача решается поиском в глубину или ширину, вторая — поиском в ширину, третья — алгоритмом Дейкстры. Это стандартные алгоритмы.

In [154]:
import networkx as nx
from pyvis.network import Network
import numpy as np

In [ ]:
# https://networkx.org/documentation/stable/reference/generators.html
# https://networkx.org/documentation/stable/reference/convert.html

V = 7 # The number of nodes.
p = 0.3 # Probability for edge creation.

G = nx.fast_gnp_random_graph(n=V, p=p, seed=42, directed=False) # NetworkX

adj_dict = nx.to_dict_of_lists(G)

adj_dict

In [97]:
# plot

# nx.draw(G, with_labels=True) # AttributeError: module 'matplotlib.cbook' has no attribute 'iterable'

nt = Network('600px', '900px', notebook=True, directed=False) # PyVis

nt.from_nx(G) # PyVis from NetworkX

nt.show('nx.html')

### Поиск в ширину

**Определение:**  

Поиск в ширину — это алгоритм, ищущий по дереву (или графу), просматривая по уровням начиная с корня.  

Алгоритм находит все узлы текущего уровня, обычно двигаясь слева направо.  
В ходе этого процесса он регистрирует все дочерние узлы, связанные с узлами на текущем уровне.  
По завершении поиска на текущем уровне, алгоритм переходит на крайний левый узел следующего уровня.  
Последним анализируется крайний правый узел самого нижнего уровня.  

**Что вам нужно знать:**  
Поиск в ширину оптимален для поиска по дереву, чья ширина превышает глубину.  
Во время хождения по дереву, алгоритм сохраняет информацию о нём в очереди.  
В связи с использованием очереди такой метод поиска потребляет больше памяти, чем поиск в глубину.  
Очередь использует память для хранения указателей.  

**Эффективность («О» большое):**  
$$O(|E| + |V|)$$  
$E$ — количество рёбер.  
$V$ — количество вершин.  

Есть ли путь из одной ноды в другую? Мы сможем утверждать, что путь есть, если мы его найдем:

In [125]:
def has_way_bfs(adj_dict, start_node, end_node):
    # ищем, есть ли путь из start_node в end_node
    # BFS - Breadth-First Search, поиск в ширину
    if start_node == end_node:
        return True
    
    visited_nodes = set() # это на случай циклов, в деревьях такого нет
    queue = [start_node]
    while queue: # пока там что-то есть
        current_node = queue.pop(0)
        neighbours_list = adj_dict.get(current_node)
        
        for node in neighbours_list:
            if node == end_node:
                return True
            if (node not in visited_nodes) and (node not in queue): # защита от циклов, в деревьях такого нет
                queue.append(node)
        
        visited_nodes.add(current_node)
        
    return False

In [126]:
# example
G = nx.fast_gnp_random_graph(
    n=7, # The number of nodes. 
    p=0.3, # Probability for edge creation.
    seed=42, 
    directed=False
)

nt = Network('600px', '900px', notebook=True, directed=False) # PyVis
nt.from_nx(G)
nt.show('nx.html')

In [128]:
has_way_bfs(
    adj_dict=nx.to_dict_of_lists(G), 
    start_node=6, 
    end_node=1
)

True

Связный ли граф (неориенторованный)? Он тогда связный, когда из одной ноды можно попасть во все другие:

In [129]:
def is_connected_bfs(adj_dict):
    # проверяем, что граф (неориенторованный) - связный
    # BFS - Breadth-First Search, поиск в ширину
    all_nodes = set(adj_dict.keys())
    
    start_node = all_nodes.pop()
    
    queue = [start_node]
    while queue: # пока там что-то есть
        current_node = queue.pop(0)
        neighbours_list = adj_dict.get(current_node)
        
        for node in neighbours_list:
            if node in all_nodes: # значит ещё не были
                all_nodes.remove(node)
                queue.append(node)
            
                if len(all_nodes) == 0:
                    return True
        
    return False

In [134]:
# example
G = nx.fast_gnp_random_graph(
    n=7, # The number of nodes. 
    p=0.2, # Probability for edge creation.
    seed=42, 
    directed=False
)

nt = Network('600px', '900px', notebook=True, directed=False) # PyVis
nt.from_nx(G)
nt.show('nx.html')

In [135]:
is_connected_bfs(
    adj_dict=nx.to_dict_of_lists(G)
)

False

А теперь кратчайший путь (в невзвешенном графе). Вот это вопросик посерьезнее.  

Хотяя... Мы всегда проходимся по порядку удаленности: сначала на 1 шаг, потом на 2 шага и т.п. Так что не пропустим кратчайший путь. Главное - как-то его сохранить (не сохранив заодно тупиковые ветки). Но если вопрос только в длине пути - это мы можем легко сделать: просто уровень вложенности надо сохранить. Вот:

In [136]:
def shortest_way_cost(adj_dict, start_node, end_node):
    # кратчайший путь из start_node в end_node (в невзвешенном графе)
    # BFS - Breadth-First Search, поиск в ширину
    costs = {n: -1 for n in adj_dict.keys()}
    
    if start_node == end_node:
        return 0
    
    costs[start_node] = 0 # чтоб не циклиться в неё
    
    queue = [start_node]
    while queue: # пока там что-то есть
        current_node = queue.pop(0)
        neighbours_list = adj_dict.get(current_node)
        for node in neighbours_list:
            if costs[node] > -1: # уже были тут
                continue
            
            costs[node] = costs[current_node] + 1
            
            # print(costs)
            
            if node == end_node:
                return costs[node]
            
            queue.append(node)
        
    return -1 # нет пути

In [137]:
# example
G = nx.fast_gnp_random_graph(
    n=7, # The number of nodes. 
    p=0.3, # Probability for edge creation.
    seed=42, 
    directed=False
)

nt = Network('600px', '900px', notebook=True, directed=False) # PyVis
nt.from_nx(G)
nt.show('nx.html')

In [139]:
shortest_way_cost(
    adj_dict=nx.to_dict_of_lists(G), 
    start_node=4, 
    end_node=2
)

2

Теоретически, так и до всех можно посчитать путь. Но нам это не обязательно, поэтому прерываем раньше.

### Поиск в глубину

**Определение:**  

Поиск в глубину — это алгоритм, ищущий по дереву (или графу) сначала в глубину начиная с корня.  

Алгоритм идёт по дереву, переходя между уровнями по левым дочерним узлам, пока не дойдёт до самого низа.  
Завершив проход по ветви, алгоритм возвращается обратно, просматривая правые дочерние узлы этой ветви. Причём, если возможно, выбирает самые левые из узлов, расположенных справа от предыдущего маршрута.  
Завершив просмотр всей ветви, алгоритм переходит к узлу, расположенному справа от корня, и снова идёт по левым дочерним узлам до самого дна.  
Последним анализируется крайний правый узел (расположенный справа от всех своих предшественников).  

**Что вам нужно знать:**  
Алгоритм оптимален для поиска по дереву, чья глубина превышает ширину.  
Для работы алгоритма используется стек.  
Поскольку стек является LIFO-структурой, ему не нужно отслеживать указатели узлов, поэтому потребляется меньше памяти, чем в случае с поиском в ширину.  
Когда алгоритм не может дальше идти по левой стороне, он начинает анализировать стек.  

**Эффективность («О» большое):**  
$$O(|E| + |V|)$$  
$E$ — количество рёбер.  
$V$ — количество вершин.

Есть ли путь из одной ноды в другую? Мы сможем утверждать, что путь есть, если мы его найдем:

In [107]:
def has_way_dfs(adj_dict, start_node, end_node):
    # ищем, есть ли путь из start_node в end_node
    # DFS - Depth-First Search, поиск в глубину
    if start_node == end_node:
        return True
    
    visited_nodes = set() # это на случай циклов, в деревьях такого нет
    
    def dfs_(current_node):
        visited_nodes.add(current_node)
        neighbours_list = adj_dict.get(current_node)
        
        for node in neighbours_list:
            print(current_node)
            if node == end_node:
                return True
            if node not in visited_nodes:
                return dfs_(node)
                
        return False
        
    return dfs_(start_node)

In [102]:
# example
G = nx.fast_gnp_random_graph(
    n=7, # The number of nodes. 
    p=0.1, # Probability for edge creation.
    seed=42, 
    directed=False
)

nt = Network('600px', '900px', notebook=True, directed=False) # PyVis
nt.from_nx(G)
nt.show('nx.html')

In [109]:
has_way_dfs(
    adj_dict=nx.to_dict_of_lists(G), 
    start_node=2, 
    end_node=6
)

2


True

Связный ли граф (неориенторованный)? Он тогда связный, когда из одной ноды можно попасть во все другие:

In [147]:
def is_connected_dfs(adj_dict):
    # проверяем, что граф (неориенторованный) - связный
    # DFS - Depth-First Search, поиск в глубину
    all_nodes = set(adj_dict.keys())
    if len(all_nodes) == 0:
        return True
        
    start_node = all_nodes.pop()
    print('start_node:', start_node)
    
    def dfs_(current_node):
        print('current_node:', current_node)
        
        neighbours_list = adj_dict.get(current_node)
        
        for node in neighbours_list:
            print('node:', node)
            if node in all_nodes: # значит ещё не были
                all_nodes.remove(node)
                if len(all_nodes) == 0:
                    print('branch return True')
                    return True
                
                dfs_(node)

    dfs_(start_node)
    
    if len(all_nodes) == 0:
        return True
    else:
        return False

In [149]:
# example
G = nx.fast_gnp_random_graph(
    n=7, # The number of nodes. 
    p=0.1, # Probability for edge creation.
    seed=42, 
    directed=False
)

nt = Network('600px', '900px', notebook=True, directed=False) # PyVis
nt.from_nx(G)
nt.show('nx.html')

In [150]:
is_connected_dfs(
    adj_dict=nx.to_dict_of_lists(G)
)

start_node: 0
current_node: 0
node: 5
current_node: 5
node: 0
node: 4
current_node: 4
node: 3
current_node: 3
node: 4
node: 5


False

### Алгоритм Дейкстры

https://python-scripts.com/dijkstras-algorithm  

В случае когда число рёбер графа примерно соответствует числу пар вершин $(|V|^2)$, простая реализация алгоритма Дейкстры будет давать асимптотику $O(|V|^2+|E|)$, что лучше варианта с приоритетной очередью.  

Если число рёбер сильно меньше количества пар вершин (такой граф называют разреженным), то вариант с приоритетной очередью будет работать быстрее. А оценка Дейкстры будет $O(∣E∣ \log{∣V∣})$

Найти кратчайший путь во взвешенном графе:

In [171]:
def shortest_way_cost_weighted(graph, start_node, end_node):
    dist = [float("inf")] * len(graph)
    
    dist[start_node] = 0
    
    all_nodes = set(adj_dict.keys())
    
    while all_nodes:
        # ищем минимальную ноду
        min_dist = float("inf")
        min_node = None
        for node in all_nodes: 
            if dist[node] < min_dist:
                min_dist = dist[node]
                min_node = node
        
        # условие для выхода
        if min_node == end_node:
            return min_dist
        
        # идём в минимальную ноду, и там работаем
        all_nodes.remove(min_node)
        neighbours_dict = graph.get(min_node)
        for neighbour_node, weight in neighbours_dict.items():
            if neighbour_node in all_nodes:
                total_dist = weight + min_dist
                if total_dist < dist[neighbour_node]:
                    dist[neighbour_node] = total_dist
                    
    return -1 # если всё обошли, и ничего не нашли

In [170]:
# example
V = 7 # number of nodes. 

adjacency_matrix = np.random.randint(
    low=-5, 
    high=5, 
    size=(V,V), 
    dtype=int
)
adjacency_matrix = np.where(adjacency_matrix > 0, adjacency_matrix, 0)
for i in range(V):
    adjacency_matrix[i,i] = 0

G = nx.from_numpy_matrix(adjacency_matrix)

nt = Network('600px', '900px', notebook=True, directed=False) # PyVis
nt.from_nx(G)
nt.show('nx.html')

In [169]:
graph = {}
for node, connections in dict(G.adj).items():
    graph[node] = {}
    for neighbour_node, weight_dict in dict(connections).items():
        graph[node][neighbour_node] = weight_dict['weight']

graph

{0: {1: 1, 5: 2, 2: 1, 4: 1},
 1: {0: 1, 2: 1, 5: 4, 4: 4},
 2: {1: 1, 0: 1, 3: 2, 4: 1, 5: 4, 6: 4},
 3: {2: 2, 4: 3, 6: 4},
 4: {2: 1, 3: 3, 0: 1, 1: 4, 4: 2, 6: 1},
 5: {0: 2, 1: 4, 2: 4, 5: 2, 6: 2},
 6: {3: 4, 4: 1, 5: 2, 2: 4, 6: 1}}

In [174]:
shortest_way_cost_weighted(
    graph=graph, 
    start_node=5, 
    end_node=2
)

3

Если хочется сам путь, то надо его хранить в виде листа (последовательности нод) для каждой посещенной ноды. И при переходе туда - складывать путь ноды, из которой пришли + путь до самой.

### Сравнение

**Сравнение поисков в ширину и в глубину**  

Выбирайте тип поиска в соответствии с размером и формой дерева:  
* Для широких, мелких деревьев используйте поиск в ширину.  
* Для глубоких, узких деревьев используйте поиск в глубину.  

**Нюансы:**  
* Поскольку поиск в ширину использует очереди для хранения информации об узлах и их детях, то он может занять больше памяти, чем доступно на вашем компьютере. (Но вам вряд ли придётся об этом беспокоиться)   
* Если применять поиск в глубину по очень глубокому дереву, то алгоритм может уходить слишком далеко вниз. Подробнее об этом читайте здесь.  
* **Поиск в ширину — циклический алгоритм**.  
* **Поиск в глубину — рекурсивный алгоритм**.  

# Деревья

На собеседованиях дерево задаётся одним из двух способов:  
* Как обычный граф, матрицей смежности или списком смежных вершин. При этом гарантируется, что граф — дерево.
* Структурой, описывающей узел бинарного дерева. На вход вашему решению передаётся корень дерева. В подобной задаче вам может потребоваться описать такую структуру. Убедитесь, что помните, как это делать в вашем языке программирования. На всякий случай потренируйтесь решать задачи без подготовленного шаблона, чтобы не допустить ошибку на собеседовании.  

Короткая шпаргалка:  
```
class TreeNode:
    def __init__(self, val):
        self.val = val
        self.left = None
        self.right = None

def traverse(root):
    ... 
```  

Обратите внимание, что связь, как правило, односторонняя: из вершины нет указателя на родительскую вершину. Построить такие указатели за один обход можно. Но нужно уточнить у интервьюера, можно ли так делать или от вас ждут решение без этого.  

У всех возможных задач на деревья общая идея решения: нужно применить поиск в глубину, то есть рекурсивный обход дерева, начиная с корня.  

Вспомним, какие бывают способы обхода деревьев:  
* _Preorder_: сначала посещаем текущую вершину, затем рассматриваем её поддеревья.  
* _Inorder_: рассматриваем левое поддерево, посещаем текущую вершину и затем рассматриваем правое поддерево. Применим только к бинарным деревьям.  
* _Postorder_: рассматриваем все поддеревья текущей вершины, затем посещаем её.  

**Задача:** Дано бинарное дерево, нужно вывести список списков значений вершин «по слоям». В каждом слое значения должны идти слева направо. Для дерева:  
```
     5
    / \
   /   \
  3     1
   \   /
    4 2 
```
нужно вернуть `[[5], [3, 1], [4, 2]]`

In [176]:
class TreeNode:
    def __init__(self, val):
        self.val = val
        self.left = None
        self.right = None
        
        
def get_layered_representation(root):
    result = []
    DFS(root, 0, result)
    return result


def DFS(node, depth, result):
    if not node:
        return
    # Т.к. мы выбрали preorder, то результат нужно увеличивать
    # не больше, чем на 1
    if depth >= len(result):
        result.append([])
    result[depth].append(node.val)
    DFS(node.left, depth + 1, result)
    DFS(node.right, depth + 1, result)

In [177]:
tree = TreeNode(5)
tree.left = TreeNode(3)
tree.right = TreeNode(1)
tree.left.right = TreeNode(4)
tree.right.left = TreeNode(2)

get_layered_representation(tree)

[[5], [3, 1], [4, 2]]

# Динамическое программирование

Динамическое программирование (для краткости будем иногда писать ДП) — это не конкретный алгоритм, а техника, которая позволяет эффективно решать довольно широкий класс задач. Такие задачи часто встречаются на собеседованиях. В этом уроке мы расскажем:  
* об основной идее динамического программирования,
* о типах задач на собеседованиях.

Начнём с классического примера. 

**Задача:** Дан массив `arr` из $N$ целых чисел. Найдите длину максимальной возрастающей подпоследовательности в этом массиве. Например, при `arr=[2, 3, 6, 4, 1, 3, 5, 4, 7]` искомая подпоследовательность — `[2, 3, 4, 5, 7]` и поэтому ответ равен 5.  

**Наивное решение:**  
1. Перебрать все возможные подпоследовательности.  
2. Для каждой из них проверить, является ли она возрастающей.  
3. Вывести максимальную длину.  

Сложность - $O(N \cdot 2^N)$  

Разберём более быстрый алгоритм, решающий эту задачу.

## Метод динамического программирования

Очень неформально можно сказать, что метод динамического программирования заключается в том, чтобы найти некоторую вспомогательную величину, которая:  
* помогает получить ответ на задачу,  
* считается понятным образом.  

Метод динамического программирования состоит в том, чтобы ответить на следующие вопросы:  
1. **Что является состоянием?** (это та вспомогательная величина, которую мы хотим найти)  
1. **Как переходить между состояниями?**  
1. **Что является начальным состоянием?**  
1. **Что является ответом на задачу?** (его надо получить из вспомогательной величины, состояния)  

В нашей ситуации:
заведём такой массив `dp` длины $N$, что в `dp[i]` будет записана длина максимальной возрастающей подпоследовательности, заканчивающаяся элементом на позиции `i`. Ответом на исходную задачу будет максимум из элементов `dp`. А чтобы вычислить `dp[i]` нужно пройтись по всем индексам `j < i`, выбрать максимум среди таких `dp[j]`, что `arr[j] < arr[i]` и прибавить к нему `1`. Если не существует таких `j`, что `j < i` и `arr[j] < arr[i]`, то `dp[i] = 1`, потому что любая последовательность из одного элемента удовлетворяет условию возрастания.

Итого:
1. **Что является состоянием?** длина максимальной возрастающей подпоследовательности, заканчивающаяся элементом на позиции `i`;
1. **Как переходить между состояниями?** Вычисляем `dp` в порядке возрастания `i` по правилу `dp[i] = max(dp[j], для j < i, arr[j] < arr[i]) + 1`.
1. **Что является начальным состоянием?** `dp[0] = 1`.
1. **Что является ответом на задачу?** - `max(dp)`

С таким решением мы получаем сложность $O(N^2)$: для каждого из $N$ элементов мы просматриваем все предыдущие, то есть $O(N)$, поэтому совокупно получается $O(N^2)$.

In [178]:
def longest_increasing_subsequence(arr):
    dp = [1] * len(arr)
    for i in range(1, len(arr)):
        for j in range(i):
            if arr[j] < arr[i]:
                dp[i] = max(dp[i], dp[j] + 1)
    return max(dp)

In [180]:
longest_increasing_subsequence([2, 3, 6, 4, 1, 3, 5, 4, 7])

5

Мы рассмотрели задачу **линейного ДП**. Это самый распространённый тип задач ДП, но мы коротко рассмотрим, какие ещё могут встретиться вам на собеседованиях.  

## Квадратичное ДП

Квадратичное ДП встречается в задачах, где множество состояний описывается не массивом, а матрицей. Рассмотрим классический пример.  

К строке можно применить одну из операций редактирования:  
* Добавить в произвольное место любой символ.  
* Заменить один символ на любой другой.  
* Удалить символ.  

_Расстояние Левенштейна_ (или редакционное расстояние) между строками `s` и `t` — это минимальное количество операций редактирования, которые нужно применить к `s`, чтобы получить `t`.  

**Задача:** Даны две строки `s` и `t`, найдите расстояние Левенштейна между ними.  

Опишем решение в терминах динамического программирования.
1. **Что является состоянием?** `dp[i][j]` — расстояние редактирования между префиксом длины `i` строки `s` и префиксом длины `j` строки `t`.  

2. **Как переходить между состояниями?**  
    * Операция добавления в первую строку ведёт к пересчёту: `dp[i][j + 1]` = `dp[i][j] + 1`  
    * Замена: `dp[i + 1][j + 1] = dp[i][j] + 1`  
    * Удаление: `dp[i + 1][j] = dp[i][j] + 1`  
    * Если же текущие символы совпадают, то никакие операции не нужны: `dp[i + 1][j + 1] = dp[i][j]`  
    
    Для каждого состояния результат будет минимумом из приведённых переходов: 
    ```
    dp[i][j] = min(dp[i - 1][j], dp[i - 1][j - 1], dp[i][j - 1]) + 1
    if s[i - 1] == t[j - 1]:
        dp[i][j] = min(dp[i][j], dp[i - 1][j - 1]) 
    ```
    В реальном коде нужно быть аккуратным в реализации и не допустить обращения к несуществующим индексам.

3. **Что является начальным состоянием?** `dp[0][0] = 0`

4. **Что является ответом на задачу?** `dp[len(s)][len(t)]`

**Задача о рюкзаке:** есть $N$ предметов, вес каждого — положительное целое число. Веса записаны в массиве `arr`. Рюкзак может выдержать содержимое не тяжелее `X`. Выберите такое подмножество предметов, чтобы их суммарный вес был максимальный, но не превосходил `X`.  

Это как раз такая задача, в которой очень важно уточнить ограничения на входные данные.  

Разберём решение за $O(N \cdot X)$, поскольку во многих реалистичных ситуациях оно будет быстрее полного перебора ($O(N \cdot 2^N)$).

Идея очень проста: будем считать, какие суммы возможно достичь, набирая по одному элементу.  
1. **Что является состоянием?** Булевский массив `dp` размерности `[N+1][X+1]`. `dp[i][j]` — достижима ли сумма `j`, если мы рассмотрели первые `i` элементов.  
2. **Как переходить между состояниями?** `dp[i][j] = True` тогда и только тогда, когда `dp[i - 1][j - arr[i - 1]] == True`  
3. **Что является начальным состоянием?** `dp[0][0] = True`
4. **Что является ответом на задачу?** Максимальный `i` такой, что `dp[N][i] == True`   

Можно уменьшить объём потребляемой памяти: нам не нужно хранить состояния для `i - 2` и раньше. Поэтому `dp[2][X+1]` будет достаточно, а переходы можно описывать как `dp[i % 2][j] = dp[(i + 1) % 2][j - arr[i - 1]]`.

# Жадные алгоритмы

Жадный алгоритм — такой, в котором на каждом шаге выбирается локально оптимальное решение. 

Одна из наиболее классических задач на эту тему: есть $N$ соревнований по спортивному программированию, `i`-е из них проходит в промежуток времени $[a_i, b_i)$. Какое максимальное количество соревнований можно выбрать, чтобы в любой момент времени проходило не более одного соревнования из этого набора?  

Чтобы решить эту задачу, нужно упорядочить соревнования по времени окончания соревнования и выбирать по следующему правилу: если предыдущее соревнование закончилось к началу рассматриваемого, то мы добавляем текущее соревнование к итоговому набору. Если нет — переходим к следующему.  

На собеседованиях жадные алгоритмы чаще всего встречаются в задачах вроде первой задачи из этого урока — прорешайте их. Ещё полезно изучить темы, связанные с **кодом Хаффмана** и **минимальным остовным деревом**.

# Задачи на реализацию

Задачи на реализацию — это не алгоритмы в привычном понимании, но они встречаются часто, поэтому про них тоже нужно поговорить.  

Как правило, в подобных задачах не появляется вопрос оптимальности. Самое сложное — придумать такой способ написать решение, чтобы не допустить ошибок.

# Структуры данных

## Динамический массив

Он выполняет те же задачи, что и обычные массивы, только позволяет менять свой размер во время выполнения.  

**Основные операции:**
* обращение по индексу элемента динамического массива (например, `arr[idx]`) всегда происходит за $O(1)$.  
* добавление элемента в конец динамического массива: $O(1)$ в среднем, $O(n)$ в худшем (если нужно проихвести реаллокацию, т.е. выделить новый кусок памяти, бОльший, чем был). Аналогично работает операция удаления.
* добавление элемента в середину динамического массива: $O(n)$ в среднем, $O(n)$ в худшем. Аналогично работает операция удаления.
* поиск по значению: $O(n)$ в среднем, $O(n)$ в худшем.  

в языке Python `list` — это динамический массив.  

Динамический массив подходит в случаях, когда нужно сохранить последовательность в некотором порядке. Можно обращаться к элементам последовательности по номеру, а не по значению. Добавлять и удалять можно только последние элементы.  

Ещё динамический массив можно использовать вместо хеш-таблицы в случае, если ключи представляют собой небольшие неотрицательные целые числа.  

Оба случая демонстрируются в примере из урока «Деревья», где мы сохраняли ключи дерева в список списков.  

## Стек (LIFO)

Стек — структура данных, которая позволяет быстро добавить элемент в конец и быстро получить последний элемент. В общем случае другие операции стек может либо не поддерживать, либо выполнять неэффективно.  

Несмотря на то, что стек выглядит очень просто, это удобная абстракция для решения некоторых задач. Например, задачу про правильные скобочные последовательности (ПСП) можно решить с помощью стека, в котором хранятся только открывающие скобки. Будем рассматривать символы строки по очереди слева направо. Если встретим открывающую скобку любого типа, поместим её в стек. Если текущая скобка — закрывающая, то она должна соответствовать скобке на вершине стека. Если это так, просто удаляем вершину из стека, а если нет, то переданная строка — не ПСП. Если мы рассмотрели все символы и после этого стек пуст — скобочная последовательность правильная.

## Очередь (FIFO)

Очередь реализует быстрые операции добавления в конец и извлечения из начала очереди.  

Хотя динамические массивы и не поддерживают быстрое извлечение элемента из начала массива, на них можно построить очередь. Для этого нужно добавить ещё один указатель, который будет показывать первый элемент в очереди. Когда надо извлечь элемент из начала, этот первый элемент возвращается, а указатель смещается на один элемент вправо.  

**Основные операции:**
* добавление элемента в конец: $O(1)$
* удаление элемента из начала: $O(1)$
* всё остальное (обращение по индексу, удаление из конца, добавление в начало, поиск по значению): $O(n)$

В Python есть `queue.Queue`  

Очередь используется в алгоритме поиска в ширину.  

## Связный список

Односвязный список — это структура данных, в которой каждый элемент имеет значение и указатель на следующий элемент. Если элемент также содержит указатель на предыдущий, то список называется двусвязным.  

В связном списке каждый элемент, кроме последнего, ссылается на следующий элемент.  

Когда в задаче говорится «дан связный список», обычно имеется ввиду, что заполнена структура списка и в функцию передаётся голова списка:  
```
class ListNode:
    def __init__(self, val):
        self.val = val
        self.next = None

def solve(head):
    # Здесь нужно написать решение предполагая,
    # что список уже заполнен
```

**Основные операции:**
* добавление элемента в конец/на следующую позицию за текущим: $O(1)$  
* удаление элемента из начала/текущего элемента: $O(1)$  
* обращение по индексу и поиск по значению: $O(n)$  

Т.е. в динамическом массиве быстро обращаться по индексу, но долго вставлять и удалять элементы на произвольные позиции. А в связном списке удобно удобно вставлять и удалять элементы куда попало, но долго искать по индексу.  

В Python отсутствует!

Список — не совсем обычная структура данных в контексте алгоритмических собеседований. С другими структурами данных часто нужно придумать, как их правильно применить и использовать реализации из стандартной библиотеки. А задачи на списки обычно формулируются явно. Разберём пример.  

**Задача:** дана голова односвязного списка, в каждом элементе которого записано целое положительное число. Нужно удалить все узлы с нечётными значениями.  

Идея решения в таких задачах очевидна, а вот с реализацией нужно быть очень аккуратным. Есть несколько советов, как сделать свою жизнь проще при решении таких задач:  
* Выделить вспомогательные функции, например, `remove_next`. Обратите внимание, что в случае односвязного списка для удаления элемента нужно знать предыдущий, только текущего недостаточно.  
* Создать дополнительную голову `fakehead`. В этом случае не нужно будет обрабатывать отдельно случай, когда голова попадает в интервал, с которым надо что-то сделать. Результирующий список будет начинаться со следующего после `fakehead` элемента.  

Приведём решение задачи с учётом этих соображений:

In [ ]:
#___________________________
class ListNode:
    def __init__(self, val):
        self.val = val
        self.next = None
#___________________________


def remove_next(node):
    to_remove = node.next
    node.next = node.next.next
    del to_remove 
    
def remove_odd(head):
    fakehead = ListNode()
    fakehead.next = head
    current = fakehead
    while current and current.next:
        if current.next.val % 2 == 1:
            remove_next(current)
        else:
            current = current.next
    return fakehead.next

Другая классическая задача: дана голова односвязного списка, нужно поменять узлы местами так, чтобы значения шли в порядке неубывания. Проще говоря, нужно отсортировать список, не копируя элементы.  

Когда речь идёт о списках, проще всего реализовать сортировку слиянием или вставками. Но последняя работает за $O(N^2)$, поэтому лучше всего выбирать сортировку слиянием.

## Хеш-таблица

Хеш-таблица — это структура данных поиска. Она позволяет быстро добавлять, искать и удалять элементы. На её основе реализованы контейнеры множество и словарь.  

**Основные операции:**  
* Добавление элемента: в среднем $O(1)$, в худшем $O(n)$;  
* Поиск элемента: в среднем $O(1)$, в худшем $O(n)$;  
* Удаление элемента: в среднем $O(1)$, в худшем $O(n)$.  

Python: `set`, `dict`  

Используйте множество, когда вам нужно быстро проверять, существует элемент в коллекции или нет.  
Используйте словарь, когда вам нужно что-то считать для элементов, которые не представляют собой непрерывный отрезок целых чисел. Например, задача: дан текст, нужно посчитать число вхождений каждого слова в текст.  

**Дополнительные замечания**  
* Не храните во множестве или ключах словарей вещественные числа. Как вы знаете, из-за особенностей представления вещественные числа нельзя сравнивать на точное равенство. Например, `0.1 + 0.2 != 0.3`, поэтому можно допустить ошибку, при которой в хеш-таблице не будет находиться элемент, хотя по смыслу он там должен быть.  
* Ключи словарей и элементы множеств изменять нельзя. Языки программирования по-разному защищают от этого. К примеру, в Python нельзя хранить во множествах или ключах словарей изменяемые структуры, поэтому нужно предварительно провести преобразования `list → tuple, set → frozenset`.  
* Для корректной работы хеш-таблиц у ключей должна существовать хеш-функция. Она уже определена для большинства классов, встроенных в языки и их стандартные библиотеки. Чтобы использовать объекты пользовательских классов как ключи в хеш-таблицах, нужно определить соответствующий метод, в Python - `__hash__`.  

## Бинарная куча и приоритетная очередь

Бинарная (двоичная) куча — структура данных, которая эффективно реализует следующие операции:
* добавить элемент,
* найти максимальный элемент,
* удалить максимальный элемент.  

При создании кучи можно задать параметры так, чтобы она поддерживала операции с минимальным элементом, а не максимальным.  

Приоритетная очередь — абстрактная структура данных, которая предоставляет те же операции. Можно сказать, что приоритетная очередь — это интерфейс, а бинарная куча — конкретная реализация этого интерфейса.  

Куча представляет собой полное бинарное дерево, в котором каждый элемент не меньше своих потомков. Таким образом, максимальный элемент всегда находится в корне, поэтому доступ к нему происходит за $O(1)$.  

> Бинарное дерево называется *полным*, если на каждом уровне, кроме последнего, существуют все вершины. А на последнем уровне вершины заполнены слева направо.

Добавление элемента происходит так: новый элемент изначально помещается на первую доступную позицию. То есть:
Если последний уровень дерева заполнен, то новый элемент становится левым потомком самой левой вершины на последнем уровне.  
В противном случае он помещается на самую левую позицию на последнем уровне.  

Дальше этот элемент сравнивается с родителем. Если родитель оказывается меньше, то они меняются местами и процесс повторяется, на этот раз уже для нового родителя элемента. Этот этап называется _просеиванием вверх_.  

Удаление максимального элемента устроено так: в корень ставится элемент с последней позиции. Дальше мы сравниваем его с потомками: если он не меньше обоих потомков, то больше ничего делать не нужно. Иначе максимальный потомок и данный элемент меняются местами. Процесс повторяется для новой позиции элемента. Этот этап называется _просеиванием вниз_.  

**Основные операции**  
Операции добавления элемента и извлечения максимального работают за время, пропорциональное высоте дерева. Поскольку оно полное, его высота всегда будет логарифмической от числа вершин, поэтому время работы этих операций — $O(\log n)$.  

* Получить значение максимального элемента: в среднем $O(1)$, в худшем $O(1)$;  
* Добавить произвольный элемент: в среднем $O(\log n)$, в худшем $O(\log n)$;  
* Удалить максимальный элемент: в среднем $O(\log n)$, в худшем $O(\log n)$;  
* Найти элемент по значению: в среднем $O(n)$, в худшем $O(n)$;  
* Удалить элемент по значению: в среднем $O(n)$, в худшем $O(n)$;  

В Python: `heapq`  

> Если вам для решения задачи потребуется стандартная структура данных, которой нет в вашем языке программирования — скажите об этом интервьюеру. Вам будет достаточно описать интерфейс этой структуры данных и решать задачу, как будто такая структура данных у вас есть.

**Когда использовать**
Когда вы решаете задачу поиска и удаления минимума или максимума в пополняемой коллекции. Классический пример — алгоритм Дейкстры. Приоритет каждой вершины — расстояние до неё от начальной вершины. На каждом шаге мы извлекаем вершину с минимальным расстоянием. Если при рассмотрении соседей текущей вершины какие-то расстояния обновились — помещаем эти вершины в очередь.  

Если в дополнение к этому вам нужно удалять произвольные элементы из коллекции, то вместо приоритетной очереди лучше использовать сбалансированное дерево поиска, о котором мы расскажем в следующем уроке.

**Дополнительные замечания**
Чтобы элементы можно было поместить в кучу, они должны быть сравнимы. То есть должны быть определены операторы или методы сравнения. Как и в случае с хеш-функциями, сравнение определено для классов стандартной библиотеки, а для пользовательских классов нужно определять методы, в Python - `__lt__`  

## Двоичное дерево поиска

Основная идея, лежащая в основе двоичного дерева поиска, заключается в том, что для любой вершины выполняется условие, которое мы сейчас опишем. Рассмотрим некоторую вершину `v` со значением `x`. В левом поддереве вершины `v` находятся вершины со значением, не превосходящим `x`, а в правом — строго больше `x`.  

Опишем алгоритм поиска некоторого значения `y`. Начиная с корня дерева, сравниваем значение в текущей вершине с `y`. Если оно равно — элемент найден, если значение в вершине меньше `y`, то идём в правое поддерево, иначе — в левое. Если соответствующего поддерева не существует, значит такого элемента нет в дереве.  

Сложность поиска соответствует глубине дерева. Если дерево построить неудачно, то сложность будет линейная. Поэтому при вставке и удалении применяется балансировка, то есть перестроение структуры дерева таким образом, чтобы глубина всегда была логарифмическая от числа вершин.  

> Ходит легенда, что на собеседованиях в крупные компании просят реализовать балансировку двоичных деревьев поиска. Это неправда, знать наизусть эти алгоритмы не нужно, достаточно знать, что они есть.  

Итак, основные операции — добавление, удаление и поиск элемента работают за $O(\log N)$. Время поиска минимального и максимального элементов зависит от реализации, но не больше $O(\log N)$. Наличие операции удаления уже существенно расширяет возможности относительно бинарной кучи.  

Но дерево поддерживает много других интересных операций. Небольшой модификацией алгоритма поиска элемента можно так же быстро находить минимальный элемент не меньше заданного. Аналогичным способом работают индексы в базах данных, которые позволяют выбирать данные по диапазону.  

В Python не входит в стандартную библиотеку, можно использовать:  
* Множество: `SortedList` из модуля `sortedcontainers`
* Словарь: `SortedDict` из модуля `sortedcontainers`

Бинарное дерево поиска — мощная структура данных, которая часто применяется на практике. Однако на собеседованиях задачи относительно несложные, поэтому обычно они решаются с помощью хеш-таблицы или двоичной кучи. Единственное исключение — задачи, в которых нужно находить статистику в скользящем окне.  

**Задача:** Дан массив целых чисел длины $N$ и число $K$. Для каждого окна длины $K$ найдите его медиану.

In [ ]:
def find_neighbours(matrix, i, j, visited_nodes):
    n = len(matrix)
    m = len(matrix[0])
    
    v = matrix[i][j]
    
    neighbours = []
    
    if ((i-1) >= 0) and (matrix[i-1][j] > v) and ((i-1, j) not in visited_nodes):
        neighbours.append((i-1, j))
    if ((i+1) < n) and (matrix[i+1][j] > v) and ((i+1, j) not in visited_nodes):
        neighbours.append((i+1, j))
        
    if ((j-1) >= 0) and (matrix[i][j-1] > v) and ((i, j-1) not in visited_nodes):
        neighbours.append((i, j-1))
    if ((j+1) < m) and (matrix[i][j+1] > v) and ((i, j+1) not in visited_nodes):
        neighbours.append((i, j+1))
        
    return neighbours

In [207]:
from typing import List


def get_longest_increasing_path(matrix: List[List[int]]) -> int:
    # your code goes here
    return 0

def read_matrix() -> List[List[int]]:
    n, m = map(int, input().split())
    matrix = []
    for i in range(n):
        matrix.append(list(map(int, input().split())))
    return matrix

matrix = read_matrix()
print(get_longest_increasing_path(matrix))

# Вероятность

**Формула суммы первых n-членов геометрической прогрессии:**  
$S_{n}=\frac{a(1-r^{n})}{1-r}$, где $n$ — количество суммируемых членов, $a$ — первый член, $r$ — знаменатель прогрессии. _Знаменатель прогрессии_ — это число, на которое нужно домножить предыдущий член, чтобы получить следующий.  

**Формула суммы первых n-членов арифметической прогрессии:**  
$S_{n}=\frac{a_1+a_n}{2} \cdot n$, где $n$ — количество суммируемых членов, $a_1$ — первый член, $a_n$ — последний член.  
$S_{n}=\frac{2a_1+d \cdot (n-1)}{2} \cdot n$, где $n$ — количество суммируемых членов, $a_1$ — первый член, $d$ — разность арифметической прогрессии. Разность арифметической прогрессии - на сколько отличаются между собой последовательные элементы (следующий - текущий или текущий - предыдущий), "шаг" прогрессии.  

вероятность одновременного наступления нескольких **независимых** событий равна _произведению_ их вероятностей. 

Если речь о **хотя бы**, то обычно проще рассуждать от противного: если хотя бы раз `А`, то _оба раза_ `не А`. А искать вероятность _оба раза_ `не А` можно через формулу независимых событий - перемножить.

**Несовместные события** не пересекаются, но и не исчерпывают всё вероятностное пространство. Или исчерпывают, но мы точно не знаем: все противоположные события несовмсестны, но не все несовместные - противоположны.  

Можно задать больше двух событий, каждое из которых не будет пересекаться ни с одним другим. В таком случае говорят, что события **попарно несовместны**, — какие два события тут ни выбрать, они будут взаимно исключать друг друга.

In [214]:
len(list(range(31,101)))

70